In [1]:
import os
from dotenv import load_dotenv
from openai import OpenAI

In [2]:
# Get the OpenAI API key from the .env file
load_dotenv('.env', override=True)
openai_api_key = os.getenv('OPENAI_API_KEY')
client = OpenAI(api_key = openai_api_key)

In [3]:
# Define a function to call the LLM
def call_llm(prompt, model="gpt-4o"):
    response = client.chat.completions.create(
        model=model,
        messages=[{"role": "user", "content": prompt}]
    )
    return response.choices[0].message.content

In [4]:
pip install --upgrade pip

Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install pdfplumber

Note: you may need to restart the kernel to use updated packages.


In [6]:
!pip install -q -U transformers==4.37.2

In [7]:
!pip install PyMuPDF

In [8]:
!pip install PyPDF2

In [9]:
!pip install pytesseract

In [10]:
import fitz  # PyMuPDF
import pytesseract
from PIL import Image

# Path to your PDF
pdf_path = "ocr test 2.pdf"

def ocr_pdf_to_text(pdf_path, output_txt="output.txt"):
    """Extract text from all pages of PDF using OCR and save to a file"""
    doc = fitz.open(pdf_path)
    all_text = []

    for i, page in enumerate(doc):
        # Convert each page to image
        pix = page.get_pixmap(dpi=300)
        img = Image.frombytes("RGB", [pix.width, pix.height], pix.samples)

        # OCR using Tesseract
        text = pytesseract.image_to_string(img, lang="eng")

        # Save per-page text
        page_header = f"\n\n===== PAGE {i+1} =====\n\n"
        all_text.append(page_header + text.strip())

    # Combine all pages into one text string
    full_text = "\n".join(all_text)

    # Save to file
    with open(output_txt, "w", encoding="utf-8") as f:
        f.write(full_text)

    return full_text
# Usage example
pdf_path = "ocr test 2.pdf"
extracted_text = ocr_pdf_to_text(pdf_path)
print(extracted_text[:2000])  # print first 2000 characters as a preview


TesseractNotFoundError: tesseract is not installed or it's not in your PATH. See README file for more information.

In [ ]:
print(extracted_text)

In [ ]:
prompt = f"""
You are a document parser. Your task is to convert the provided PDF text and form data
into structured JSON.

Rules:

1. Include all fields in the output JSON, even if their value is empty or null.
Understand the file structure.
2. Input may contain checkbox fields, understand how it is represented.
For checkboxes or radio buttons:
   -Include only the items that are selected (marked with '✔' or similar).
   - If no items in a checkbox/radio group are selected, set the field to null.
   - Do NOT output boolean true/false for unselected items.
   - Extract only the items that are marked as selected, indicated by a leading '✔' or similar mark.
    - Ignore items without any selection mark.
    - If no items are selected in a checklist, set it to null.
    - Include only the items that are marked as selected, indicated by a leading '✔', or 'n' or similar mark.
  - Do NOT output boolean true/false flags for items.
  - If no items are selected in a checklist, set that checklist field to null.
  - if checkbox is under different column in table, keep it under that column name. You have follow the structure.

3. Split full names into 'FirstName' and 'LastName' if possible.
4. Group logically related fields together.
5. Output valid JSON only. No explanation, no extra text.
6.Extract data if it is split into two coloumns and/or numbered columns data

Here are the filled form fields (checkboxes, radios, etc.):

{extracted_text}

Here is the extracted text from the PDF:

{extracted_text}
"""
prompt_ = prompt.format(extracted_text=extracted_text)

In [ ]:
# Get response from LLM
# print(prompt_)
import re
from pprint import pprint
response_content = call_llm(prompt_)
pattern = r'<text>(.*?)</text>'
match = re.search(pattern, response_content, re.DOTALL)
if match:
    resp = json.loads(match.group(1))
    pprint(resp)
    print(resp.keys())
else:
    print(response_content)